# Dataset Surtur

In [7]:
import pandas as pd 
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500


import numpy as np
from glob import glob as globlin
from bs4 import BeautifulSoup
import re
from urlextract import URLExtract
import statistics
import os
import csv
import onemillion

from tqdm import tqdm
tqdm.pandas()

import signal
from contextlib import contextmanager

from xgboost import XGBClassifier

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot
import plotly.express as px


@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        return 'timeout error'
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)


def raise_timeout(signum, frame):
    raise TimeoutError

## Reading New Data

In [3]:
def read_data(main_path):
    """reading the benign samples"""
    dataframe_array = []
    benign_data_paths = globlin(main_path)
    for path in benign_data_paths:
        dataframe_array.append(pd.read_csv(path))
        #print(pd.read_csv(path).columns)
    return pd.concat(dataframe_array)

In [5]:
dataset_surtur = read_data('./data_construction/4 - final_data/*.csv').drop(columns=['Unnamed: 0.1'])
df_mal_sup = read_data('./data_construction/supp_malicious_1.csv')

/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [ ]:
df_mal_sup = df_mal_sup.drop(columns=['Label'])

In [ ]:
dataset_surtur = dataset_surtur.drop(columns=['status', 'https.1'])

In [ ]:
df_mal_sup.columns = ['url', 'has_IP_in_url', 'number_subdomains', 'hostname',
       'length_hostname', 'ratio_digits_url', 'having_@_in_url',
       'ratio_digits_hostname', 'number_underscores', 'tld', 'url_len',
       'https', 'who_is', 'content', 'label']

df_mal_sup.columns

In [ ]:
dataset_surtur = pd.concat([dataset_surtur, df_mal_sup], axis=0, sort=False).drop_duplicates()
dataset_surtur.head()

In [ ]:
dataset_surtur['label'].value_counts()

## Extracting JS

In [ ]:
def get_js(content, supp=False):
    tags_of_interest = [
        '<script type="text/javascript">', 
        '<script>'
    ]
    
    soup = BeautifulSoup(content, 'html.parser')
    js = soup.find_all('script')
    complete_js = ''
    for tag in js:
        for tag_int in tags_of_interest:
            if tag_int in str(tag):
                complete_js += str(tag).replace(tag_int, '').replace(
                    '</script>', '')
    return complete_js.encode('utf-8')

In [ ]:
dataset_surtur['js'] = dataset_surtur['content'].progress_apply(lambda content: get_js(str(content)))

## JS Length

In [ ]:
dataset_surtur['js_len'] = dataset_surtur['js'].progress_apply(lambda js: len(js))

In [ ]:
ex_string_thing = dataset_surtur[dataset_surtur['js_len']==0].iloc[1]['content']

## Counting Script tag references in html page 

In [ ]:
def script_references(js):
    return js.count('<script')/2

In [ ]:
dataset_surtur['js_ref'] = dataset_surtur['content'].progress_apply(
    lambda content: script_references(str(content)))

## Getting array lengths from JS 

In [ ]:
def get_max_array_length(js):
    array_lengths = re.findall('\(([^\)]+)\)', js)
    if array_lengths == []:
        return 0
    return max([len(i) for i in array_lengths])

def get_avg_array_length(js):
    array_lengths = re.findall('\(([^\)]+)\)', js)
    if array_lengths == []:
        return 0
    return statistics.mean([len(i) for i in array_lengths])

In [ ]:
dataset_surtur['js_array_len_avg'] = dataset_surtur['js'].progress_apply(
    lambda js: get_avg_array_length(str(js)))
dataset_surtur['js_array_len_max'] = dataset_surtur['js'].progress_apply(
    lambda js: get_max_array_length(str(js)))

## Content Length

In [ ]:
dataset_surtur['content_len'] = dataset_surtur['content'].progress_apply(
    lambda x: len(str(x)))

## Number of Function calls

In [ ]:
dataset_surtur['js'][1]

In [ ]:
def get_func_calls(js):
    full_paren = len(re.findall("\(([^\)]+)\)", js))
    empty_paren = len(js.split('()'))
    return full_paren + empty_paren

In [ ]:
dataset_surtur['num_js_func_calls'] = dataset_surtur['js'].progress_apply(
    lambda x: get_func_calls(str(x)))

## Suspicious Function Count

In [ ]:
def get_sus_js_function_count(js):
    function_list = [
        'setcookie', 'getcookie', 'createxmlhttprequest', 'unescape',
        'document.write', 'element.appendchild', 'dateobject.togmtstring',
        'new activexobject', 'document.createelement', 'getappname',
        'getuseragent', 'window.setinterval', 'window.settimeout',
        'location.assign', 'location.replace', 'eval()', 'string.indexof',
        'string.fromcharcode', 'charat', 'split',
        'string.charcodeat', 'document.writeln', 'document.appendchild',
        'innerhtml', 'insertAdjacentHTML', 'outerhtml', 
    ]

    split_js = js.replace('\n',' ').split(' ')
    counter = 0
    for element in split_js:
        if any(m_function in element.lower() for m_function in function_list):
            counter += 1

    return counter

In [ ]:
dataset_surtur['malicious_func_count'] = dataset_surtur['js'].progress_apply(
    lambda js: get_sus_js_function_count(str(js)))

## Get total and external URL count in content

In [ ]:
# def find_urls(string, ext_count):
#     # with timeout(1):
#     regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
#     url = re.findall(regex, string)
#     if ext_count:
#         return len(set(url))
#     return len(url)


# # Test Code
# string = 'My Profile: https://auth.geeksforgeeks.org/user/Chinmoy%20Lenka/articles in the portal of http://www.geeksforgeeks.org/'
# print("Urls: ", find_urls(string, False))

In [ ]:
def find_urls(string, ext_count):
    try:
        extractor = URLExtract()
        urls = extractor.find_urls(string)
        if ext_count: 
            return len(set(urls))
        return len(urls)
    except:
        return 10000

string_ex = 'My Profile: https://auth.geeksforgeeks.org/user/Chinmoy%20Lenka/articles in the portal of http://www.geeksforgeeks.org/'
    
print(find_urls(string_ex, False))

In [ ]:
dataset_surtur['total_url_count'] = dataset_surtur['content'].progress_apply(
    lambda js: find_urls(str(js), False)
)

In [ ]:
dataset_surtur['ext_url_count'] = dataset_surtur['content'].progress_apply(
    lambda js: find_urls(str(js), True)
)

In [ ]:
dataset_surtur.columns

## More Lexical Features

In [ ]:
dataset_surtur['num_semicolons'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count(';'))
dataset_surtur['num_zeros'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('0'))
dataset_surtur['num_spaces'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('%20'))
dataset_surtur['num_hyphens'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('-'))
dataset_surtur['num_@s'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('@'))
dataset_surtur['num_queries'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('?'))
dataset_surtur['num_ampersands'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('&'))
dataset_surtur['num_equals'] = dataset_surtur['url'].progress_apply(lambda url: str(url).count('='))

## Safe Browsing API Judgement

In [11]:
with open('./safebrowsingkey.txt') as f:
    api_key = f.read()

In [ ]:
import requests

key = 'your key here'
URL = "https://sb-ssl.google.com/safebrowsing/api/lookup?client=api&apikey={key}&appver=1.0&pver=3.0&url={url}"


def is_safe(url):
    try:
        response = requests.get(URL.format(key=api_key, url=url))
        return response.text != 'malware'
    except:
        return 'unknown'


print(is_safe('http://addonrock.ru/Debugger.js/'))  # prints False
print(is_safe('http://google.com'))  # prints True


dataset_surtur['google_is_safe'] = dataset_surtur['url'].progress_apply(lambda url: is_safe(url))

  0%|          | 0/105492 [00:00<?, ?it/s]

True
True


 29%|██▉       | 30556/105492 [2:09:00<9:30:34,  2.19it/s] 

## Retrieving Domain names and their lengths

In [ ]:
dataset_surtur['domain'] = dataset_surtur['hostname'].progress_apply(
    lambda hostname: '.'.join(str(hostname).split('.')[-2:]))

In [ ]:
dataset_surtur['domain_len'] = dataset_surtur['domain'].progress_apply(lambda domain: len(domain))

## Presence in Alexa domains

In [ ]:
CONFIG = {
    'domain_lists': [
        {
            'name': "alexa",
            'output_file_path': "alexa.csv",
            'url': "http://s3.amazonaws.com/alexa-static/top-1m.csv.zip"
        }, {
            'name': "cisco umbrella",
            'output_file_path': "cisco.csv",
            'url': "http://s3-us-west-1.amazonaws.com/umbrella-static/" +
                   "top-1m.csv.zip"
        }
    ]
}

# Cache our top 1 million known domains
DEFAULT_CACHE_LOCATION = '~/.onemillion'
cache_location = os.path.expanduser(DEFAULT_CACHE_LOCATION)

# O(n) to read Csv file rows into set 
def read_onemillion_data():
    """Check if the given domain is in a top on million list."""
    # TODO: parse the registered domain out of the domain parameter

    # keep track of the highest (nearest to 1) rank for the given domain
    highest_rank = None

    # see if the given domain is in the up-to-date domain lists
    domains = set()
    for domain_list in CONFIG['domain_lists']:
        # open the domain list as a CSV
        with open(os.path.join(cache_location, domain_list['output_file_path']), 'r') as domain_csv:
            domain_reader = csv.reader(domain_csv)
            for row in domain_reader:
                domains.add(row[1])
    return domains


domains_onemillion = read_onemillion_data()

# O(1) Function Run time
def domain_checker(domain):
    domain = domain.lower()
    if domain in domains:
        return True
    return False

In [ ]:

o = onemillion.OneMillion()

def check_if_in_onemillion(domain):
    if domain in domains_onemillion:
        return 1
    else:
        return 0
    
    
dataset_surtur['is_in_alexa'] = dataset_surtur['domain'].progress_apply(
    lambda domain: check_if_in_onemillion(domain))    

## One hot encoding

In [ ]:
def column_adjustor(dataset_column):
    unique_values = dataset_column.unique()
    return dataset_column.progress_apply(lambda x: np.where(unique_values == x)[0][0])

In [ ]:
dataset_surtur = dataset_surtur.dropna()

In [ ]:
dataset_surtur['who_is'] = column_adjustor(dataset_surtur['who_is'])
dataset_surtur['https'] = column_adjustor(dataset_surtur['https'])
dataset_surtur['tld'] = column_adjustor(dataset_surtur['tld'])

dataset_surtur['label'] = dataset_surtur['label'].progress_apply(
    lambda label: 1 if 'bad' in label else 0)

# Checkpoint 

In [ ]:
# dataset_surtur.to_csv('./dataset_surtur.csv', index=False)

In [8]:
dataset_surtur = pd.read_csv('./dataset_surtur.csv')
dataset_surtur.head()

,content,has_IP_in_url,having_@_in_url,hostname,https,label,length_hostname,number_subdomains,number_underscores,ratio_digits_hostname,ratio_digits_url,tld,url,url_len,who_is,js,js_len,js_ref,js_array_len_avg,js_array_len_max,content_len,num_js_func_calls,malicious_func_count,total_url_count,ext_url_count,num_semicolons,num_zeros,num_spaces,num_hyphens,num_@s,num_queries,num_ampersands,num_equals,domain,domain_len
0,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,0.0,islandvolleyballclub.com,0,0,24.0,0.0,0.0,0.0,0.0,0,http://islandvolleyballclub.com/venues.html,43.0,0,b'',0,0.0,0.0,0,7813,1,0,15,14,0,0,0,0,0,0,0,0,islandvolleyballclub.com,24
1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",0.0,0.0,www.indiosoftware.com,0,0,21.0,1.0,0.0,0.0,0.0,0,http://www.indiosoftware.com/,29.0,0,b'\nfunction getCookie(c_name) { // Local func...,1103,0.5,17.2,69,1556,23,4,2,2,0,0,0,0,0,0,0,0,indiosoftware.com,17
2,"<html>\n<head>\n<meta content=""text/html; char...",0.0,0.0,naturalfilters.bizland.com,0,0,26.0,1.0,0.0,0.0,0.0,0,http://naturalfilters.bizland.com/aquarain/,43.0,0,"b'\nfunction myopen(filename,windowname,proper...",133,0.5,30.0,30,26533,4,0,40,28,0,0,0,0,0,0,0,0,bizland.com,11
3,<!DOCTYPE HTML>\n\n<html><!-- InstanceBegin te...,0.0,0.0,www.rosepath.com,0,0,16.0,1.0,0.0,0.0,0.0,0,http://www.rosepath.com,23.0,0,"b'qm_create(0,false,0,0,false,false,false,fals...",53,1.5,41.0,41,6465,2,0,5,5,0,0,0,0,0,0,0,0,rosepath.com,12
4,could not fetch content,0.0,0.0,www.plannedparrothood.com,0,0,25.0,1.0,0.0,0.0,0.0,0,http://www.plannedparrothood.com/,33.0,0,b'',0,0.0,0.0,0,23,1,0,0,0,0,0,0,0,0,0,0,0,plannedparrothood.com,21


In [9]:
# dataset_surtur = dataset_surtur[dataset_surtur['content']!='could not fetch content']

In [10]:
dataset_surtur['label'].value_counts()

0    61083
1    44409
Name: label, dtype: int64

## Normalizing 

In [ ]:
to_keep = dataset_surtur.drop(
    columns=['url', 'content', 'hostname', 'js', 'label']).columns[::-1]

In [ ]:
to_keep = dataset_surtur.drop(
    columns=['url', 'content', 'hostname', 'js', 'label', 'domain']).columns[::-1]
# 'total_url_count', 'ext_url_count'

x = dataset_surtur[dataset_surtur['label'] == 1][to_keep].copy()  #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.values)
df_to_plot_malicious = pd.DataFrame(x_scaled)
df_to_plot_malicious.columns = dataset_surtur[to_keep].columns
df_to_plot_malicious['label'] = 1.0

x = dataset_surtur[dataset_surtur['label'] == 0][to_keep].copy()  #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.values)
df_to_plot_benign = pd.DataFrame(x_scaled)
df_to_plot_benign.columns = dataset_surtur[to_keep].columns
df_to_plot_benign['label'] = 0.0

normalized_per_class_dataset = pd.concat([df_to_plot_malicious, df_to_plot_benign], axis=0)

x = dataset_surtur[to_keep].copy()  #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.values)
dataset_surt_norm = pd.DataFrame(x_scaled)
dataset_surt_norm.columns = dataset_surtur[to_keep].columns
dataset_surt_norm['label'] = dataset_surtur['label'].values

## Outlier removal 

In [ ]:
# from scipy import stats

# num_df_surt = dataset_surt_norm.select_dtypes(include=["number"])
# cat_df_surt = dataset_surt_norm.select_dtypes(exclude=["number"])

# # idx = np.all(stats.zscore(num_df_surt) < 3, axis=1)

# # dataset_surt_norm_cleaned = pd.concat([num_df_surt.loc[idx], 
# #                                    cat_df_surt.loc[idx]], axis=1)

# Q1 = num_df_surt.quantile(0.10)
# Q3 = num_df_surt.quantile(0.80)
# IQR = Q3 - Q1
# idx = ~((num_df_surt < (Q1 - 1.5 * IQR)) | (num_df_surt > (Q3 + 1.5 * IQR))).any(axis=1)
# dataset_surt_norm_cleaned = pd.concat([num_df_surt.loc[idx], 
#                                     cat_df_surt.loc[idx]], axis=1)

In [ ]:
# len(dataset_surt_norm_cleaned)

In [ ]:
dataset_surtur['label'].value_counts()

In [ ]:
# indexes_to_drop = dataset_surtur.nlargest(15000, 'js_len')['js_len'].index
# dataset_surtur = dataset_surtur.drop(indexes_to_drop, axis='index')

## Feature Analysis

In [ ]:
features = dataset_surtur.drop(columns=['url', 'content', 'hostname', 'js', 
                                        'label']).columns[::-1] #, 'total_url_count', 'ext_url_count']).columns[::-1]
df_to_plot = dataset_surtur

n_bins = 40

fig, axs = plt.subplots(3, 6, figsize=(20,20))

# We can set the number of bins with the `bins` kwarg
feature_counter = 0
for i in range(len(axs)):
    for j in range(len(axs[i])): 
        current_feature = df_to_plot[features[feature_counter]]
        axs[i, j].hist(current_feature[df_to_plot['label']==0.0], n_bins, fc=(0, 1, 0, 0.5))
        axs[i, j].hist(current_feature[df_to_plot['label']==1.0], n_bins, fc=(1, 0, 0, 0.5))
        axs[i, j].set_title(f'Feature: {features[feature_counter]}')
        if features[feature_counter] == 'asdf':
            axs[i, j].set_ylim([0, current_feature.value_counts().iloc[0]]) 
        else:
            axs[i, j].set_ylim([0, max(current_feature.value_counts())]) 
        feature_counter += 1
    if feature_counter > 14:
        break

plt.show()

## Plotly Analysis

In [ ]:
dataset_surtur.columns

In [ ]:
# js_array_len_avg

In [ ]:
feature = 'is_in_alexa'
# feature = dataset_surt_norm.columns[2]
dataset_to_plot = dataset_surtur

good_filter = dataset_to_plot[feature][dataset_to_plot['label'] == 0]#.progress_apply(lambda x: roundup(x))
bad_filter = dataset_to_plot[feature][dataset_to_plot['label'] == 1]#.progress_apply(lambda x: roundup(x))


# bad_filter = bad_filter[bad_filter!=bad_filter.max()]

trace1 = go.Histogram(
    x=good_filter,
    name='Benign',
    yaxis='y2'

)

trace2 = go.Histogram(
    x=bad_filter,
    name='Malicious',
    yaxis='y2'
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)

fig['layout'].update(height = 700, width = 1000, title = f'Feature: {feature}',xaxis=dict(tickangle=-90))
iplot(fig)

# Machine Learning 

## Standard Set

### Train-test-split

In [ ]:
X = dataset_surtur.drop(columns = ['label', 'content', 'hostname', 'url', 
                                   'js', 'domain']).astype('float32') # 'total_url_count', 'ext_url_count'
# X = np.array(X).reshape([len(X), 1])
y = dataset_surtur['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### XGBoost

In [ ]:
X.columns[11]

In [ ]:
xgboost_model = XGBClassifier()
xgboost_model.fit(X_train, y_train)

In [ ]:
y_pred = xgboost_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['benign', 'malicious'], digits=4))

In [ ]:
X.columns[20]

In [ ]:
xgboost_model.feature_importances_

In [16]:
# X_test.iloc[29664]

In [ ]:
for index, prediction in enumerate(y_pred):
    if prediction != y_test.values[index] and y_test.values[index] == 1:
        print(index)

## Checking misclassified samples

In [ ]:
# for index, value in enumerate(y_pred):
#     if value != y_test[index]:
#         pass
#         #print(index)

In [ ]:
dataset_surtur.iloc[3]

### Random Forest

In [ ]:
rfc_model = RandomForestClassifier(max_depth=2, random_state=0)
rfc_model.fit(X_train, y_train)

In [ ]:
y_pred = rfc_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, 
                            target_names=['benign', 'malicious'], 
                            digits=4))

### SVM

In [ ]:
param_grid = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}

svm_model = svm.SVC(verbose=3)
clf = GridSearchCV(svm_model, param_grid, n_jobs=-1, cv=3, verbose=3)

clf.fit(X_train, y_train)

In [ ]:
print(clf.best_params_)
print(clf.best_estimator_)

In [ ]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, 
                            target_names=['benign', 'malicious'], 
                            digits=4))

## Neural Network

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, AveragePooling2D, Flatten, Dropout
from tensorflow.python.client import device_lib

model = Sequential([
#     Conv2D(filters=6,
#            kernel_size=(3, 3),
#            activation='relu',
#            input_shape=X_train.shape),
#     Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
#     AveragePooling2D(),
#     Flatten(),
    Dense(units=1210, activation='relu'),
    Dense(units=841, activation='relu'),
    Dense(units=1, activation='softmax')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_history = model.fit(X_train, y_train, #validation_data=(X_valid_NN, y_valid_NN), 
                          epochs=5, batch_size=32, verbose=True)

In [ ]:
import seaborn as sn

plt.subplots(figsize=(30,20))
corrMatrix = X.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
def histogram_intersection(a, b):
    v = np.minimum(a, b).sum().round(decimals=1)
    return v

X.corr(method=histogram_intersection)

## Feature Selection - Chi Squared

In [ ]:
X_new = SelectKBest(chi2, k=8).fit_transform(X, y)
X_new.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=42)

### XGBoost

In [ ]:
xgboost_model = XGBClassifier()
xgboost_model.fit(X_train, y_train)

In [ ]:
y_pred = xgboost_model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['benign', 'malicious'], digits=4))